In [ ]:
import serial
import time
import threading
import os


import glob
import pymeasure
import pyvisa
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from time import sleep
from scipy.signal import argrelextrema
from tqdm.notebook import tqdm
plt.style.use(['science','nature','notebook'])


from datetime import datetime
from sklearn import preprocessing



#my_path = os.path.abspath(r'C:\Users\KuChris\Desktop\laser')
my_path = os.getcwd()


In [ ]:
##Reg
ITLA_NOERROR=0x00
ITLA_EXERROR=0x01
ITLA_AEERROR=0x02
ITLA_CPERROR=0x03
ITLA_NRERROR=0x04
ITLA_CSERROR=0x05
ITLA_ERROR_SERPORT=0x01
ITLA_ERROR_SERBAUD=0x02

REG_Nop=0x00
REG_DevTyp=0x01
REG_Mfgr=0x02
REG_Model=0x03
REG_Serial=0x04
REG_MFGDate=0x05
REG_Release=0x06
REG_RelBack=0x07
REG_Gencfg=0x08
REG_AeaEac=0x09
REG_AeaEa=0x0A
REG_AeaEar=0x0B
REG_Iocap=0x0D
REG_Ear=0x10
REG_Dlconfig=0x14
REG_Dlstatus=0x15
REG_Channel=0x30
REG_Power=0x31
REG_Resena=0x32
REG_Grid=0x34
REG_Fcf1=0x35
REG_Fcf2=0x36
REG_LF1=0x40
REG_LF2=0x41
REG_Oop=0x42
REG_CTemp=0x43
REG_Opsl=0x50
REG_Opsh=0x51
REG_Lfl1=0x52
REG_Lfl2=0x53
REG_Lfh1=0x54
REG_Lfh2=0x55
REG_Currents=0x57
REG_Temps=0x58
REG_Ftf=0x62
REG_Mode=0x90
REG_PW=0xE0
REG_Csweepsena=0xE5
REG_Csweepamp=0xE4
REG_Cscanamp=0xE4
REG_Cscanon=0xE5
REG_Csweepon=0xE5
REG_Csweepoffset=0xE6
REG_Cscanoffset=0xE6
REG_Cscansled=0xF0
REG_Cscanf1=0xF1
REG_Cscanf2=0xF2
REG_CjumpTHz=0xEA
REG_CjumpGHz=0xEB
REG_CjumpSled=0xEC
REG_Cjumpon=0xED
REG_Cjumpoffset=0xE6

READ=0
WRITE=1
latestregister=0
tempport=0
raybin=0
queue=[]
maxrowticket=0

_error=ITLA_NOERROR
seriallock=0

In [ ]:
##functions
def stripString(input):
    outp=''
    input=str(input)
    teller=0
    while teller<len(input) and ord(input[teller])>47:
        outp=outp+input[teller]
        teller=teller+1
    return(int(outp))

def checksum(byte0,byte1,byte2,byte3):
    bip8=(byte0&0x0f)^byte1^byte2^byte3
    bip4=((bip8&0xf0)>>4)^(bip8&0x0f)
    return bip4

def Send_command(sercon,byte0,byte1,byte2,byte3):
    sercon.write([byte0,byte1,byte2,byte3])

def Receive_response(sercon):
    global _error,queue
    reftime=time.perf_counter()
    while sercon.inWaiting()<4:
        if time.perf_counter()>reftime+0.25:
            _error=ITLA_NRERROR
            return(0xFF,0xFF,0xFF,0xFF)
        time.sleep(0.0001)
    try:
        byte0=ord(sercon.read(1))
        byte1=ord(sercon.read(1))
        byte2=ord(sercon.read(1))
        byte3=ord(sercon.read(1))
    except:
        print ('problem with serial communication. queue[0] =',queue)
        byte0=0xFF
        byte1=0xFF
        byte2=0xFF
        byte3=0xFF
    if checksum(byte0,byte1,byte2,byte3)==byte0>>4:
        _error=byte0&0x03
        return(byte0,byte1,byte2,byte3, hex(byte0),hex(byte1),hex(byte2),hex(byte3))
    else:
        _error=ITLA_CSERROR
        return(hex(byte0),hex(byte1),hex(byte2),hex(byte3))

def ITLA(sercon,register,data,rw):
    global latestregister
    lock=threading.Lock()
    lock.acquire()
    global queue
    global maxrowticket
    rowticket=maxrowticket+1
    maxrowticket=maxrowticket+1
    queue.append(rowticket)
    lock.release()
    while queue[0]!=rowticket:
        rowticket=rowticket
    if rw==0:
        byte2=int(data/256)
        byte3=int(data-byte2*256)
        latestregister=register
        Send_command(sercon,int(checksum(0,register,byte2,byte3))*16,register,byte2,byte3)
        test=Receive_response(sercon)
        b0=test[0]
        b1=test[1]
        b2=test[2]
        b3=test[3]
        if (b0&0x03)==0x02:
            test=AEA(sercon,b2*256+b3)
            lock.acquire()
            queue.pop(0)
            lock.release()
            return test
        lock.acquire()
        queue.pop(0)
        lock.release()
        return b2*256+b3
    else:
        byte2=int(data/256)
        byte3=int(data-byte2*256)
        Send_command(sercon,int(checksum(1,register,byte2,byte3))*16+1,register,byte2,byte3)
        test=Receive_response(sercon)
        lock.acquire()
        queue.pop(0)
        lock.release()
        return(test[2]*256+test[3])


def AEA(sercon,bytes):
    outp=''
    while bytes>0:
        Send_command(sercon,int(checksum(0,REG_AeaEar,0,0))*16,REG_AeaEar,0,0)
        test=Receive_response(sercon)
        outp=outp+chr(test[2])
        outp=outp+chr(test[3])
        bytes=bytes-2
    return outp

def ITLAConnect(port,baudrate):
    global CoBrite
    reftime=time.perf_counter()
    connected=False
    try:
        conn = serial.Serial(port, baudrate , timeout=1)
    except serial.SerialException:
        return(ITLA_ERROR_SERPORT)
    baudrate2=4800
    while baudrate2<115200:
        ITLA(conn,REG_Nop,0,0)
        if ITLALastError()!=ITLA_NOERROR:
            #go to next baudrate
            if baudrate2==4800:baudrate2=9600
            elif baudrate2==9600: baudrate2=19200
            elif baudrate2==19200: baudrate2=38400
            elif baudrate2==38400:baudrate2=57600
            elif baudrate2==57600:baudrate2=115200
            conn.close()
            conn = serial.Serial(port,baudrate2 , timeout=1)
        else:
            return(conn)
    conn.close()
    return(ITLA_ERROR_SERBAUD)

def ITLALastError():
    return(_error)

s = serial.Serial(port="COM4", baudrate=9600, bytesize=8, timeout=1, stopbits=serial.STOPBITS_ONE)
#s = ITLAConnect(port="COM4",baudrate=9600)

In [ ]:
#Connection
if not s.isOpen() == True:
    s.open()
s.close()
s = serial.Serial(port="COM4", baudrate=9600, bytesize=8, timeout=1, stopbits=serial.STOPBITS_ONE)

if ITLA(s,REG_Nop,0,0) == 16:
   print('Laser is working.')
else:
   print('Laser is not working.')

##Connectiontest

#s = ITLAConnect(port='COM4',baudrate=9600)


In [ ]:
print('Specification:')

#device type
dt = ITLA(s,REG_DevTyp,0,0)
print('device type= ' + dt)

#Manufacturer
man = ITLA(s,REG_Mfgr,0,0)
print('Manufacturer= ' + man)

#Model
mod = ITLA(s,REG_Model,0,0)
print('Model= ' + mod)

#SerNo
serno = ITLA(s,REG_Serial,0,0)
print('SerNo= ' + serno)

#MFGdate
mfgd = ITLA(s,REG_MFGDate,0,0)
print('MFGDate= ' + mfgd)

#firmware
firm = ITLA(s,REG_Release,0,0)
print('Firmware= ' + firm)

#firmwareback
firmb = ITLA(s,REG_RelBack,0,0)
print('Firmware backwards= ' + firmb)

#General module config
gencfg = ITLA(s,REG_Gencfg,0,0)
print('General module config= ' + str(gencfg))

# #AeaEac Ea Ear
# reg = ITLA(s,REG_AeaEar,0,0)
# print(reg)

#io information
# ioinf = ITLA(s,REG_Iocap,0,0)
# print(ioinf)

#ask for lowest and highest power
powerl = ITLA(s,REG_Opsl,0,0)
powerh =ITLA(s,REG_Opsh,0,0)
print('powerl= '+str(powerl*0.01)+'dBm' +' , ' 'powerh= ' + str(powerh*0.01)+'dBm')

#ask for Lowest and highest Fre
lfl = (ITLA(s,REG_Lfl1,0,0) + ITLA(s,REG_Lfl2,0,0)*0.0001)
lfh = (ITLA(s,REG_Lfh1,0,0) + ITLA(s,REG_Lfh2,0,0)*0.0001)
print('lfl= '+str(lfl)+'Thz' +' , ' 'lfh= ' + str(lfh)+'Thz')


In [ ]:
##Setting
Power = 1000
Grid = 1000
Channel = 1
Fre = 191.3
finefre = 0

In [ ]:
print('Current states:')
#Reset
ITLA(s,REG_Resena,0,1)

#Temp
print('Temp = ' + str(ITLA(s,REG_CTemp,0,0)/100)+'C')

# #Currents
# print('Currents = '+ ITLA(s,REG_Currents,0,0))

#Set PWR level
ITLA(s,REG_Power,Power,1)
#p = ITLA(s,REG_Oop,0,0)
print('Power = ' + str(Power*0.01) + 'dBm')

#Set Grid
ITLA(s,REG_Grid,Grid,1)
print('Grid = ' + str(Grid))

#Set Channel
ITLA(s,REG_Channel,Channel,1)
print('Channel = ' + str(Channel))

#Set Frequency
Fre1 = int(Fre)
Fre2 = int(round((float(Fre) - Fre1) * 10000, 4))
ITLA(s,REG_Fcf1,Fre1,1)
ITLA(s,REG_Fcf2,Fre2,1)
fr = ITLA(s,REG_LF1,0,0)+ITLA(s,REG_LF2,0,0)*0.0001
print('Frequency = ' + str(fr) + 'THz')

#Set Fine tune Frequency
ITLA(s,REG_Ftf,finefre,1)
print('Fine tune Frequency = ' + str(ITLA(s,REG_Ftf,0,0)))


In [ ]:
ITLA(s,REG_Resena,8,1)

In [ ]:
#hp8612c
rm = pyvisa.ResourceManager()
rm.list_resources()
hp86120c = rm.open_resource('GPIB0::23::INSTR')
hp86120c.timeout =30000
hp86120c.read_termination = '\n'
hp86120c.write_termination = '\n'
hp86120c.query('*IDN?')

In [ ]:
# #power and wav
# hp86120c.write(':INIT:CONT OFF')
# hp86120c.write('*WAI')
# hp86120c.query(':MEAS:ARR:POW:WAV? MIN')
# #hp86120c.write(':CONF:ARR:POW MIN')
# #hp86120c.write(':INIT:IMM')

# hp86120c.write('*WAI')
# power = float(hp86120c.query('FETC:SCAL:POW?'))
# wave = float(hp86120c.query('FETC:SCAL:POW:WAV?'))
# wavehz = float(hp86120c.query('FETC:SCAL:POW:FREQuency?'))
# print('Power = ' + str(power) + ' dBm')
# print('Wavelenght nm = ' + str(wave*1e9) + ' nm')
# print('Wavelenght hz = ' + str(wavehz*1e-12) + ' Hz')

# hp86120c.write('*WAI')

# #StoN
# hp86120c.write('CALC3:SNR:STAT ON')
# #Set first wavelength as noise reference
# hp86120c.write('CALC3:SNR:REF:WAV MIN')
# #Query signal-to-noise values
# snr = hp86120c.query(':CALC3:DATA? POW')
# print('SNR = ' + str(snr) + ' dBm')
# hp86120c.write('CALC3:SNR:STAT OFF')


#power and wav
hp86120c.write(':INIT:CONT OFF')
hp86120c.write('*WAI')
hp86120c.query(':MEAS:ARR:POW:WAV? 0.01')
#hp86120c.write(':CONF:ARR:POW MIN')
#hp86120c.write(':INIT:IMM')

hp86120c.write('*WAI')
power = float(hp86120c.query('FETC:SCAL:POW?'))
wave = float(hp86120c.query('FETC:SCAL:POW:WAV?'))
wavehz = float(hp86120c.query('FETC:SCAL:POW:FREQuency?'))
print('Power = ' + str(power) + ' dBm')
print('Wavelenght nm = ' + str(wave*1e9) + ' nm')
print('Wavelenght hz = ' + str(wavehz*1e-12) + ' Hz')

hp86120c.write('*WAI')
#StoN
hp86120c.write('CALC3:SNR:STAT ON')
#Set first wavelength as noise reference
hp86120c.write('CALC3:SNR:REF:WAV 1550')
#Query signal-to-noise values
snr = hp86120c.query(':CALC3:DATA? POW')
print('SNR = ' + str(snr) + ' dBm')

In [ ]:
float(snr)

In [ ]:
#set point
hp86120c.write(':CALCulate1:TRANsform:FREQuency:POINts MAX')
print(hp86120c.query('CALCulate1:TRANsform:FREQuency:POINTs?'))
data = hp86120c.query(':CALCulate1:DATA?')

In [ ]:
hp86120c.write('CALC2:PWAV:STAT ON')
hp86120c.query(':CALCulate2:DATA? POW')
#hp86120c.query(':CALCulate2:DATA?')

In [ ]:
data2 = data.split(',')

powerdata=[]

for datas in data2:
    powerdata.append(float(datas))
    
print(len(powerdata))

powerdata2 = np.array(powerdata).reshape(-1,1)

print(max(powerdata))
print(min(powerdata))

# norm = []
# for unnorm in powerdata:
#     normed = (unnorm-min(powerdata))/(max(powerdata)-min(powerdata))
#     norm.append(normed)

In [ ]:
dedBe = 1
if dedBe == 1:
    de = 9.65
if dedBe == 0:
    de = 0

normalized= preprocessing.normalize(powerdata2,axis=0,norm='max')
#plt.plot(normalized)
plt.plot(np.linspace(1650,1270,len(powerdata)), 5*np.log10(normalized)+float(hp86120c.query(':CALCulate2:DATA? POW'))+de,linewidth=0.5)

max(5*np.log10(normalized))+float(hp86120c.query(':CALCulate2:DATA? POW'))+de

In [ ]:
# powerdata = pd.DataFrame({'Power': data})
# power.to_csv(r'C:\Users\KuChris\Desktop\powerdata.csv')

In [ ]:
# Function to convert from mW to dBm
def mW2dBm(mW):
    return 10.*np.log10(mW)

# Function to convert from dBm to mW
def dBm2mW(dBm):
    return 10**((dBm)/10.)

In [ ]:
n = 1
n1000=int((196-191.3)//(1000/10000.0))+2
snrdata=[]
#n10000=int((196-191.3)//(10000/10000.0))+1

ITLA(s,REG_Grid,1000,1)
ITLA(s,REG_Channel,1,1)

sleep(1)
ITLA(s,REG_Resena,8,1)
for n in tqdm(range(1,n1000+1), desc='Progress'):
    while True:
        print(ITLA(s,REG_Oop,0,0))
        sleep(1)
        if ITLA(s,REG_Oop,0,0) < Power:
            sleep(1)
        elif ITLA(s,REG_Oop,0,0) > 10000:
            sleep(1)
        else:
            for i in range(1):
                #power and wav
                hp86120c.write(':INIT:CONT OFF')
                hp86120c.write('*WAI')
                hp86120c.query(':MEAS:ARR:POW:WAV? 0.1')
                #hp86120c.write(':CONF:ARR:POW MIN')
                #hp86120c.write(':INIT:IMM')

                hp86120c.write('*WAI')
                power = float(hp86120c.query('FETC:SCAL:POW?'))
                wave = float(hp86120c.query('FETC:SCAL:POW:WAV?'))
                wavehz = float(hp86120c.query('FETC:SCAL:POW:FREQuency?'))
                print('Power = ' + str(power) + ' dBm')
                print('Wavelenght nm = ' + str(wave*1e9) + ' nm')
                print('Wavelenght hz = ' + str(wavehz*1e-12) + ' Hz')

                hp86120c.write('*WAI')
                #StoN
                hp86120c.write('CALC3:SNR:STAT ON')
                #Set first wavelength as noise reference
                hp86120c.write('CALC3:SNR:REF:WAV 1550')
                #Query signal-to-noise values
                snr = hp86120c.query(':CALC3:DATA? POW')
                print('SNR = ' + str(snr) + ' dBm')
                snrdata.append(float(snr))

                hp86120c.write('CALC3:SNR:STAT OFF')

                hp86120c.write('*WAI')
                hp86120c.write(':CALCulate1:TRANsform:FREQuency:POINts MAX')
                print('Number of data = ' + str(hp86120c.query('CALCulate1:TRANsform:FREQuency:POINTs?')))
                data = hp86120c.query(':CALCulate1:DATA?')

                hp86120c.write('*WAI')
                hp86120c.write('CALC2:PWAV:STAT ON')
                realpower = float(hp86120c.query(':CALCulate2:DATA? POW'))
                print('read power = ' + str(realpower))
                

                data2 = data.split(',')
                powerdata=[]
                for datas in data2:
                    powerdata.append(float(datas))
                
                uncorrecteddata = pd.DataFrame({'Power': powerdata})
                uncorrecteddata.to_csv(r"C:\Users\KuChris\Desktop\osa\86120c\uncorrected\data_{:02d}_{:02d}.csv".format(n,i))

                powerdata2 = np.array(powerdata).reshape(-1,1)

                dedBe = 1
                if dedBe == 1:
                    de = 9.65
                if dedBe == 0:
                    de = 0

                normalized= preprocessing.normalize(powerdata2,axis=0,norm='max')
                
                correcteddata = pd.DataFrame({'Power': (5*np.log10(normalized.reshape(1,-1))+realpower+de)[0]})
                correcteddata.to_csv(r"C:\Users\KuChris\Desktop\osa\86120c\corrected\data_{:02d}_{:02d}.csv".format(n,i))

                plt.plot(np.linspace(1650,1270,len(powerdata)), 5*np.log10(normalized)+float(hp86120c.query(':CALCulate2:DATA? POW'))+de,linewidth=0.5)
                plt.show()
            powerdata.clear()
            if n == n1000:
                break
            else:
                ITLA(s,REG_Channel,n+1,1)
                sleep(2)
                if ITLA(s,REG_Oop,0,0) > 10000 or ITLA(s,REG_Oop,0,0) < Power:
                    break

allsnrdata = pd.DataFrame({'SNR': snrdata})
allsnrdata.to_csv(r"C:\Users\KuChris\Desktop\osa\86120c\allsnrdata.csv")
snrdata.clear()
ITLA(s,REG_Resena,0,1)

#max(5*np.log10(normalized))+float(hp86120c.query(':CALCulate2:DATA? POW'))+de


In [ ]:
ITLA(s,REG_Resena,0,1)

In [ ]:
joined_files = os.path.join(r"C:\Users\KuChris\Desktop\osa\86120c\corrected", "*.csv")
joined_list = glob.glob(joined_files)

In [ ]:
dfs = []
for file in joined_list:
    df = pd.read_csv(file)
    dfs.append(df)

In [ ]:
import scipy.constants as sci

hzi = 181.6915
nmf =(sci.c/(hzi*1e12))*1e9

hzf = 236.0584
nmi =(sci.c/(hzf*1e12))*1e9

nmf, nmi

def hztonm(x):
    a = (sci.c/(x*1e12))*1e9
    return a


In [ ]:
joined_files = os.path.join(r"C:\Users\KuChris\Desktop\osa\86120c\corrected", "*.csv")
joined_list = glob.glob(joined_files)
dfs = []
smsr= []
for file in joined_list:
    df = pd.read_csv(file)
    dfs.append(df)

s=0
x = np.arange(hzi,hzf,3.613378e-3)
y = dfs[s]['Power']
xnm = np.sort(hztonm(x))[::-1]

# plt.figure()
# #plt.xlim(191,197)
# plt.plot(x,y)
# max_yhz = max(y)
# max_xhz = x[y.argmax()]
# print(max_xhz, max_yhz)


plt.figure()
#plt.xlim(1520,1570)
plt.plot(xnm,y)
plt.tight_layout()
max_ynm = max(y)
max_xnm = xnm[y.argmax()]
print(max_xnm, max_ynm)

plt.figure()
dfs[s]['nm'] = xnm
dfs[s]=dfs[s][(dfs[s].nm <= 1570) & (dfs[s].nm >= 1520)]

dfs[s]['max'] = dfs[s]['Power'][(dfs[s]['Power'].shift(1) < dfs[s]['Power']) & (dfs[s]['Power'].shift(-1) < dfs[s]['Power'])]
dfs[s]['max'] = dfs[s].iloc[argrelextrema(dfs[s]['Power'].values, np.greater_equal, order=50)[0]]['Power']
smsrvalue = dfs[s]['max'].dropna().nlargest(2).values[0]-dfs[s]['max'].dropna().nlargest(2).values[1]
plt.plot(dfs[s]['nm'],dfs[s]['Power'])
plt.scatter(dfs[s]['nm'], dfs[s]['max'], c='r')
plt.tight_layout()
print(smsrvalue)



In [ ]:
joined_files = os.path.join(r"C:\Users\KuChris\Desktop\osa\86120c\corrected", "*.csv")
joined_list = glob.glob(joined_files)
dfs = []
smsr= []
for file in joined_list:
    df = pd.read_csv(file)
    dfs.append(df)

plt.figure(dpi=120)
for s in range (0,47):
#s=0
    x = np.arange(hzi,hzf,3.613378e-3)
    y = dfs[s]['Power']
    xnm = np.sort(hztonm(x))[::-1]

    dfs[s]['nm'] = xnm
    dfs[s]=dfs[s][(dfs[s].nm <= 1570) & (dfs[s].nm >= 1520)]

    dfs[s]['max'] = dfs[s]['Power'][(dfs[s]['Power'].shift(1) < dfs[s]['Power']) & (dfs[s]['Power'].shift(-1) < dfs[s]['Power'])]
    dfs[s]['max'] = dfs[s].iloc[argrelextrema(dfs[s]['Power'].values, np.greater_equal, order=50)[0]]['Power']
    smsrvalue = dfs[s]['max'].dropna().nlargest(2).values[0]-dfs[s]['max'].dropna().nlargest(2).values[1]
    smsr.append(smsrvalue)
    plt.plot(dfs[s]['nm'],dfs[s]['Power'])
    plt.scatter(dfs[s]['nm'], dfs[s]['max'], c='r')
    plt.tight_layout()

snrdf = pd.read_csv(os.path.join(r'C:\Users\KuChris\Desktop\osa\86120c', 'allsnrdata.csv'))
plt.figure(dpi=120)
plt.plot(snrdf['SNR'],label='SNR')

plt.plot(np.arange(len(smsr)),smsr,label='SMSR')
plt.scatter(np.arange(len(smsr)),smsr, c='r')
plt.title('SMSR vs 48 channels')
plt.xlabel('Channel')
plt.ylabel('dBm')
plt.legend()
plt.tight_layout()

